# Импорты и Настройки

In [ ]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
import sys
from variational_inference import VariationalEnsemble

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

sys.path.append("../methods/")


# Фиксируем сид для воспроизводимости тестов библиотеки
torch.manual_seed(42)
np.random.seed(42)

# Демонстрация обучения

In [ ]:
def run_test():
    # Данные
    def generate_data(n_samples=150):
        torch.manual_seed(42)
        X = torch.linspace(-3, 3, n_samples).unsqueeze(1)
        y = torch.sin(X) * X + torch.randn_like(X) * 0.1
        return X, y

    X_train, y_train = generate_data()
    loader = DataLoader(TensorDataset(X_train, y_train), batch_size=64, shuffle=True)

    # Модель
    base_model = nn.Sequential(
        nn.Linear(1, 64), nn.ReLU(), nn.Linear(64, 64), nn.ReLU(), nn.Linear(64, 1)
    )

    print("Инициализация...")
    ensemble = VariationalEnsemble(
        model=base_model, prior_sigma=1.0, learning_rate=0.01, auto_convert=True
    )

    print("Обучение...")
    #history = ensemble.fit(loader, epochs=800, kl_weight=0.05, verbose=True)
    ensemble.fit(loader, epochs=800, kl_weight=0.05, verbose=True)

    # Тест
    X_test = torch.linspace(-6, 6, 300).unsqueeze(1)
    mean, std = ensemble.predict(X_test, n_samples=100)
    sampled_models = ensemble.sample_models(n_models=5)

    # Визуализация
    plt.figure(figsize=(12, 6))
    plt.scatter(X_train, y_train, c="black", s=10, label="Train Data", zorder=5)

    # Uncertainty
    upper = (mean + 2 * std).flatten()
    lower = (mean - 2 * std).flatten()
    plt.fill_between(
        X_test.flatten(),
        lower,
        upper,
        color="blue",
        alpha=0.2,
        label="Uncertainty (95%)",
    )
    plt.plot(X_test, mean, c="blue", linewidth=2, label="Mean Prediction")

    # Samples
    for i, model in enumerate(sampled_models):
        with torch.no_grad():
            y_sample = model(X_test)
        plt.plot(
            X_test,
            y_sample,
            c="red",
            alpha=0.3,
            linewidth=1,
            label="Sampled Model" if i == 0 else None,
        )

    plt.title("Bayesian Neural Network Test")
    plt.legend()
    plt.ylim(-4, 4)
    plt.grid(True, alpha=0.3)
    plt.show()

Начинаем обучение...
Epoch 0 | Loss: 3401.9 | KL: 761.1 | Noise Sigma: 0.129
Epoch 500 | Loss: 356.2 | KL: 376.3 | Noise Sigma: 0.361
Epoch 1000 | Loss: 287.5 | KL: 291.2 | Noise Sigma: 0.556
Epoch 1500 | Loss: 264.3 | KL: 255.0 | Noise Sigma: 0.728
Обучение завершено.


In [ ]:
if __name__ == "__main__":
    run_test()